# Imports

In [3]:
# %% Imports y configuración
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime
from pathlib import Path
import json

# Agregar el directorio raíz al path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..'))

# Imports del proyecto
from model_ddp.utils.sistem_fun import (
    load_config,
    get_data_path,
    get_artifact_path,
    get_report_path,
    create_experiment_id,
    ensure_directories
)

from model_ddp.simulations.gaussian_simulator import (
    RegressionSimulator,
    SimulationConfig,
    RBFKernel,
    MaternKernel,
    PeriodicKernel,
    TransformationFunctions
)

# Modelos
from model_ddp.models.LSBP_normal_v1 import LSBPNormal

# Metricas y graficas 
from model_ddp.fit.metrics import regression_metrics
from model_ddp.graphics.plots_regression import plot_regression_analysis

config=load_config()

# Flujo 

- Datos: Se guardaran todo en una carpeta. Por tanto todo tratamiento debe ir previo a un experimento.
- Experimentos: Cada modelo quedara en una carpeta con id, por tanto se crean versiones para cada uno.

In [4]:
#Parametros Iniciales
NOMBRE_EJECUCION = "model_lsbp_001"
SIM_REAL = "simulation"

In [5]:
##################################################
# Crear carpeta de guardado 
##################################################
data_path = get_data_path(config, SIM_REAL, "output")
carpeta_datos = data_path / f"{NOMBRE_EJECUCION}"
carpeta_datos.mkdir(parents=True, exist_ok=True)

##################################################
# Simulacion o data real
##################################################
# Configuración de la simulación
sim_config = SimulationConfig(
    n_samples=50,
    n_features=5,
    x_range=(0.0, 10.0),
    noise_std=0.3,
    random_state=234
)

# Definir kernel (RBF)
kernel = RBFKernel(
    length_scale=2.0,
    variance=1.0
)

# Definir transformación (regresión lineal)
coefficients = np.array([2.5, -1.8, 0.9, 1.2, -0.5])
intercept = 3.0

transformation = TransformationFunctions.linear(
    coefficients=coefficients,
    intercept=intercept
)

# Crear simulador
simulator = RegressionSimulator(
    config=sim_config,
    kernel=kernel,
    transformation=transformation
)

# Generar datos
print("Generando datos...")
X, Y = simulator.simulate()

print("✓ Datos generados exitosamente")
print(f"\nEstadísticas de X:")
print(f"  Shape: {X.shape}")
print(f"  Media por feature: {X.mean(axis=0)}")
print(f"  Std por feature: {X.std(axis=0)}")
print(f"\nEstadísticas de Y:")
print(f"  Shape: {Y.shape}")
print(f"  Media: {Y.mean():.4f}")
print(f"  Std: {Y.std():.4f}")
print(f"  Min: {Y.min():.4f}")
print(f"  Max: {Y.max():.4f}")

##################################################
# Transformar a data frame 
##################################################
datos = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(sim_config.n_features)])
datos['Y'] = Y

##################################################
# Guardar data frame  
##################################################
csv_filename = f"{carpeta_datos}/_data.csv"
datos.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

Generando datos...
✓ Datos generados exitosamente

Estadísticas de X:
  Shape: (50, 5)
  Media por feature: [ 0.02670214  0.83881283  0.33954869  0.1927579  -0.56643199]
  Std por feature: [0.44819103 0.71500407 0.41225002 0.4855196  0.50575982]

Estadísticas de Y:
  Shape: (50,)
  Media: 2.4036
  Std: 1.7811
  Min: -0.4483
  Max: 6.4302
✓ Datos guardados en CSV: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\data\simulaciones\model_lsbp_001/_data.csv


## Experimento I 

In [6]:
# Parámetros de ejecución
CARACTERISTICAS = "Test inicial: GP con kernel RBF + transformación lineal"
EXPERIMENT_ID = create_experiment_id("lsbp_001")

### Modelo

In [9]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO LSBPNormal...")
print("="*60)

# Crear instancia del modelo
lsbp_model = LSBPNormal(
    y=datos["Y"].values,
    X=datos.drop(columns=["Y"]).values,
    H=15,                     # Número inicial de clusters truncados
    verbose=True              # Mostrar progreso
)

# Ejecutar MCMC
trace = lsbp_model.run(
    iterations=50,          # Iteraciones totales
    burnin=10               # Burn-in
)

print("\n" + "="*60)
print("LSBP COMPLETADO")
print("="*60)


EJECUTANDO LSBPNormal...
✓ Usando aceleracion C++
⚠️ Error en update_alpha C++: module 'model_ddp.models.LSBP_normal_v1.lsbp_cpp' has no attribute 'update_alpha_batch'

LSBP COMPLETADO


### Guardado Modelo

In [10]:
##################################################
#  Guardar Modelo  (SEGUN EXPERIMENTO)
##################################################

# Crear carpetas para guardar
artifact_path = get_artifact_path(config, SIM_REAL)
carpeta_modelo = artifact_path / f"{EXPERIMENT_ID}"
carpeta_modelo.mkdir(parents=True, exist_ok=True)

print("\n" + "="*60)
print("GUARDANDO MODELO Y RESULTADOS...")
print("="*60)

# 1. Guardar el modelo completo (objeto LSBPNormal)
model_file = carpeta_modelo / "lsbp_model.pkl"
with open(model_file, 'wb') as f:
    pickle.dump(lsbp_model, f)
print(f"✓ Modelo guardado: {model_file}")

# 2. Guardar solo las trazas (más ligero)
trace_file = carpeta_modelo / "trace.pkl"
with open(trace_file, 'wb') as f:
    pickle.dump(trace, f)
print(f"✓ Trazas guardadas: {trace_file}")

# 3. Guardar resumen posterior
summary = lsbp_model.get_posterior_summary()
summary_file = carpeta_modelo / "posterior_summary.json"
with open(summary_file, 'w') as f:
    # Convertir tuplas a listas para JSON
    summary_json = {k: {'mean': v[0], 'std': v[1]} for k, v in summary.items()}
    json.dump(summary_json, f, indent=2)
print(f"✓ Resumen posterior guardado: {summary_file}")

# 4. Guardar metadatos del experimento
metadata = {
    'experiment_id': EXPERIMENT_ID,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'model_type': 'LSBPNormal',
    'data_shape': {
        'n': lsbp_model.n,
        'p': lsbp_model.p
    },
    'hyperparameters': {
        'H_initial': 15,
        'iterations': 50,
        'burnin': 10,
        'n_grid': lsbp_model.n_grid
    },
    'priors': {
        'mu_prior': (lsbp_model.mu_mu, lsbp_model.tau_mu_inv),
        'mu0_prior': (lsbp_model.m0, lsbp_model.s02),
        'kappa0_prior': (lsbp_model.alpha_kappa, lsbp_model.beta_kappa),
        'a0_prior': (lsbp_model.alpha_a, lsbp_model.beta_a),
        'b0_prior': (lsbp_model.alpha_b, lsbp_model.beta_b),
        'psi_prior': (lsbp_model.mu_psi, lsbp_model.tau_psi_inv)
    },
    'final_stats': {
        'H_final': lsbp_model.H,
        'n_clusters_mean': summary['n_clusters'][0],
        'n_clusters_std': summary['n_clusters'][1]
    },
    'acceptance_rates': {
        'alpha': np.mean(lsbp_model.mh_acceptance['alpha'][-100:]) if lsbp_model.mh_acceptance['alpha'] else 0,
        'psi': np.mean(lsbp_model.mh_acceptance['psi'][-100:]) if lsbp_model.mh_acceptance['psi'] else 0,
        'kappa0': np.mean(lsbp_model.mh_acceptance['kappa0'][-100:]) if lsbp_model.mh_acceptance['kappa0'] else 0,
        'a0': np.mean(lsbp_model.mh_acceptance['a0'][-100:]) if lsbp_model.mh_acceptance['a0'] else 0
    }
}

metadata_file = carpeta_modelo / "metadata.json"
with open(metadata_file, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✓ Metadatos guardados: {metadata_file}")

# 5. Guardar información de normalización 
normalization_file = carpeta_modelo / "normalization.pkl"
normalization_data = {
    'y_mean': lsbp_model.y_mean,
    'y_std': lsbp_model.y_std,
    'X_mean': lsbp_model.X_mean,
    'X_std': lsbp_model.X_std
}
with open(normalization_file, 'wb') as f:
    pickle.dump(normalization_data, f)
print(f"✓ Datos de normalización guardados: {normalization_file}")

print("\n" + "="*60)
print(f"MODELO GUARDADO EN: {carpeta_modelo}")
print("="*60)


GUARDANDO MODELO Y RESULTADOS...
✓ Modelo guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\lsbp_001_20251224_005012\lsbp_model.pkl
✓ Trazas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\lsbp_001_20251224_005012\trace.pkl
✓ Resumen posterior guardado: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\lsbp_001_20251224_005012\posterior_summary.json
✓ Metadatos guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\lsbp_001_20251224_005012\metadata.json
✓ Datos de normalización guardados: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\lsbp_001_20251224_005012\normalization.pkl

MODELO GUARDADO EN: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\artefact\simulaciones\models\lsbp_001_20251224_005012


### Predicciones, grafica de Predicciones y guardado

In [11]:
##################################################
#  FIT  (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO PREDICCIONES...")
print("="*60)

# Hacer predicciones con el modelo entrenado
y_pred_mean, y_pred_std = lsbp_model.predict_mean(
    X_new=datos.drop(columns=["Y"]).values,
    n_samples=100
)

y_true = datos["Y"].values

# Calcular métricas
metrics = regression_metrics(y_true, y_pred_mean)

print("\n📊 MÉTRICAS DE AJUSTE:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

# Guardar predicciones completas
predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred_mean,
    'y_pred_std': y_pred_std,
    'residual': y_true - y_pred_mean,
    'residual_std': (y_true - y_pred_mean) / y_pred_std  # Residuos estandarizados
})
predictions_file = carpeta_reportes / "predictions.csv"
predictions_df.to_csv(predictions_file, index=False)
print(f"✓ Predicciones guardadas: {predictions_file}")

##################################################
# Gráficas de Diagnóstico (SEGUN EXPERIMENTO)
##################################################
print("\n" + "="*60)
print("GENERANDO GRÁFICAS...")
print("="*60)
# Crear carpeta para gráficas
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred_mean, "Training Set")
]

plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="LSBP Normal"
)

print(f"✓ Gráficas guardadas en: {carpeta_graficas}")


GENERANDO PREDICCIONES...


AttributeError: 'LSBPNormal' object has no attribute 'predict_mean'

### Otros analisis

In [8]:
##################################################
# Analisis especificas del modelo 
##################################################

##################################################
# Gráficas Adicionales Específicas de LSBP
##################################################
print("\n" + "="*60)
print("GENERANDO GRÁFICAS ESPECÍFICAS DE LSBP...")
print("="*60)

# 1. Evolución del número de clusters
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(trace['n_clusters'], linewidth=1.5)
ax.set_xlabel('Iteración (post burn-in)', fontsize=12)
ax.set_ylabel('Número de Clusters Activos', fontsize=12)
ax.set_title('Evolución del Número de Clusters', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(carpeta_graficas / "evolucion_clusters.png", dpi=300, bbox_inches='tight')
plt.close()

# 2. Traza de hiperparámetros
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Trazas de Hiperparámetros', fontsize=14, fontweight='bold')

hyperparams = [
    ('mu', 'μ (Intercepto stick-breaking)'),
    ('mu0', 'μ₀ (Media base)'),
    ('kappa0', 'κ₀ (Precisión relativa)'),
    ('a0', 'a₀ (Shape σ²)'),
    ('b0', 'b₀ (Scale σ²)'),
    ('n_clusters', 'Número de Clusters')
]

for idx, (param, label) in enumerate(hyperparams):
    ax = axes[idx // 3, idx % 3]
    ax.plot(trace[param], linewidth=1, alpha=0.8)
    ax.set_xlabel('Iteración', fontsize=10)
    ax.set_ylabel(label, fontsize=10)
    ax.grid(alpha=0.3)
    ax.set_title(f'{label}\nMedia: {np.mean(trace[param]):.3f}', fontsize=10)

plt.tight_layout()
plt.savefig(carpeta_graficas / "trazas_hiperparametros.png", dpi=300, bbox_inches='tight')
plt.close()

# 3. Distribución posterior de hiperparámetros
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Distribución Posterior de Hiperparámetros', fontsize=14, fontweight='bold')

for idx, (param, label) in enumerate(hyperparams):
    ax = axes[idx // 3, idx % 3]
    ax.hist(trace[param], bins=30, density=True, alpha=0.7, edgecolor='black')
    ax.axvline(np.mean(trace[param]), color='red', linestyle='--', 
               linewidth=2, label=f'Media: {np.mean(trace[param]):.3f}')
    ax.axvline(np.median(trace[param]), color='blue', linestyle='--', 
               linewidth=2, label=f'Mediana: {np.median(trace[param]):.3f}')
    ax.set_xlabel(label, fontsize=10)
    ax.set_ylabel('Densidad', fontsize=10)
    ax.legend(fontsize=8)
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(carpeta_graficas / "distribucion_posterior_hiperparametros.png", dpi=300, bbox_inches='tight')
plt.close()

# 4. Tasas de aceptación MH
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Tasas de Aceptación Metropolis-Hastings', fontsize=14, fontweight='bold')

mh_params = ['alpha', 'psi', 'kappa0', 'a0']
mh_labels = ['α (Intercepto clusters)', 'ψ (Decaimiento kernel)', 
             'κ₀ (Precisión)', 'a₀ (Shape)']

for idx, (param, label) in enumerate(zip(mh_params, mh_labels)):
    ax = axes[idx // 2, idx % 2]
    
    if lsbp_model.mh_acceptance[param]:
        # Calcular tasa de aceptación móvil (ventana de 50)
        acceptance = np.array(lsbp_model.mh_acceptance[param])
        window = 50
        moving_avg = np.convolve(acceptance, np.ones(window)/window, mode='valid')
        
        ax.plot(moving_avg, linewidth=1.5, alpha=0.8)
        ax.axhline(0.234, color='red', linestyle='--', linewidth=2, 
                   label='Óptimo (0.234)')
        ax.set_xlabel('Iteración', fontsize=10)
        ax.set_ylabel('Tasa de Aceptación (ventana=50)', fontsize=10)
        ax.set_title(f'{label}\nMedia: {np.mean(acceptance):.3f}', fontsize=10)
        ax.legend(fontsize=8)
        ax.grid(alpha=0.3)
        ax.set_ylim([0, 1])

plt.tight_layout()
plt.savefig(carpeta_graficas / "tasas_aceptacion_mh.png", dpi=300, bbox_inches='tight')
plt.close()

# 5. Intervalos de credibilidad en predicciones
fig, ax = plt.subplots(figsize=(12, 6))

# Ordenar por y_true para visualización más clara
sort_idx = np.argsort(y_true)
y_true_sorted = y_true[sort_idx]
y_pred_sorted = y_pred_mean[sort_idx]
y_std_sorted = y_pred_std[sort_idx]

ax.scatter(range(len(y_true)), y_true_sorted, alpha=0.6, s=30, 
           label='Observado', color='blue')
ax.plot(range(len(y_pred_mean)), y_pred_sorted, 'r-', 
        linewidth=2, label='Predicción (media)')
ax.fill_between(range(len(y_pred_mean)),
                y_pred_sorted - 1.96 * y_std_sorted,
                y_pred_sorted + 1.96 * y_std_sorted,
                alpha=0.3, color='red', label='IC 95%')

ax.set_xlabel('Observación (ordenada)', fontsize=12)
ax.set_ylabel('Y', fontsize=12)
ax.set_title('Predicciones con Intervalos de Credibilidad 95%', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(carpeta_graficas / "intervalos_credibilidad.png", dpi=300, bbox_inches='tight')
plt.close()

print(f"✓ Todas las gráficas guardadas en: {carpeta_graficas}") 


GENERANDO GRÁFICAS ESPECÍFICAS DE LSBP...
✓ Todas las gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\lsbp_001_20251223_183659


### Registrar experimento

In [9]:

##################################################
# Registrar Experimento
##################################################
print("\n" + "="*60)
print("REGISTRANDO EXPERIMENTO...")
print("="*60)

# Preparar información del experimento
experiment_data = {
    'experiment_id': EXPERIMENT_ID,
    'nombre': NOMBRE_EJECUCION,
    'tipo': SIM_REAL,
    'descripcion': f"""
Experimento: {CARACTERISTICAS}

**Modelo**: LSBPNormal (Logit Stick-Breaking Process con kernel Normal)

**Características**:
- Datos: n={lsbp_model.n}, p={lsbp_model.p}
- Clusters iniciales: {15}
- Clusters finales: {lsbp_model.H}
- Iteraciones MCMC: {50}
- Burn-in: {10}
- Grid kernel: {lsbp_model.n_grid} puntos

**Priors utilizados**:
- μ ~ N({lsbp_model.mu_mu}, {lsbp_model.tau_mu_inv})
- μ₀ ~ N({lsbp_model.m0}, {lsbp_model.s02})
- κ₀ ~ Gamma({lsbp_model.alpha_kappa}, {lsbp_model.beta_kappa})
- a₀ ~ Gamma({lsbp_model.alpha_a}, {lsbp_model.beta_a})
- b₀ ~ Gamma({lsbp_model.alpha_b}, {lsbp_model.beta_b})
    """,
    'configuracion': f"""
```yaml
model:
  type: LSBPNormal
  H_initial: 15
  iterations: 50
  burnin: 10
  n_grid: {lsbp_model.n_grid}

data:
  n_samples: {lsbp_model.n}
  n_features: {lsbp_model.p}
  source: {carpeta_datos / '_data.csv'}
```
    """,
    'resultados': f"""
**Métricas de Ajuste**:
- MSE: {metrics['mse']:.6f}
- RMSE: {metrics['rmse']:.6f}
- MAE: {metrics['mae']:.6f}
- R²: {metrics['r2']:.6f}
- MAPE: {metrics['mape']:.6f}%

**Estadísticas Posteriores**:
- Número de clusters (media): {summary['n_clusters'][0]:.2f} ± {summary['n_clusters'][1]:.2f}
- μ (posterior): {summary['mu'][0]:.3f} ± {summary['mu'][1]:.3f}
- μ₀ (posterior): {summary['mu0'][0]:.3f} ± {summary['mu0'][1]:.3f}
- κ₀ (posterior): {summary['kappa0'][0]:.3f} ± {summary['kappa0'][1]:.3f}
- a₀ (posterior): {summary['a0'][0]:.3f} ± {summary['a0'][1]:.3f}
- b₀ (posterior): {summary['b0'][0]:.3f} ± {summary['b0'][1]:.3f}

**Tasas de Aceptación MH**:
- α: {metadata['acceptance_rates']['alpha']:.3f}
- ψ: {metadata['acceptance_rates']['psi']:.3f}
- κ₀: {metadata['acceptance_rates']['kappa0']:.3f}
- a₀: {metadata['acceptance_rates']['a0']:.3f}
    """,
    'archivos': f"""
**Modelo y Datos**:
- Modelo: `{model_file.relative_to(get_project_root())}`
- Trazas: `{trace_file.relative_to(get_project_root())}`
- Normalización: `{normalization_file.relative_to(get_project_root())}`
- Metadatos: `{metadata_file.relative_to(get_project_root())}`

**Métricas**:
- Métricas (JSON): `{metrics_file.relative_to(get_project_root())}`
- Métricas (CSV): `{metrics_csv.relative_to(get_project_root())}`
- Predicciones: `{predictions_file.relative_to(get_project_root())}`

**Gráficas**:
- Análisis de regresión: `{(carpeta_graficas / 'analisis_regresion.png').relative_to(get_project_root())}`
- Residuos: `{(carpeta_graficas / 'residuos_regresion.png').relative_to(get_project_root())}`
- Evolución clusters: `{(carpeta_graficas / 'evolucion_clusters.png').relative_to(get_project_root())}`
- Trazas hiperparámetros: `{(carpeta_graficas / 'trazas_hiperparametros.png').relative_to(get_project_root())}`
- Distribución posterior: `{(carpeta_graficas / 'distribucion_posterior_hiperparametros.png').relative_to(get_project_root())}`
- Tasas aceptación: `{(carpeta_graficas / 'tasas_aceptacion_mh.png').relative_to(get_project_root())}`
- Intervalos credibilidad: `{(carpeta_graficas / 'intervalos_credibilidad.png').relative_to(get_project_root())}`
    """
}
registry_file = save_experiment_metadata(config, experiment_data)
print(f"✓ Experimento registrado en: {registry_file}")




REGISTRANDO EXPERIMENTO...


NameError: name 'get_project_root' is not defined